In [5]:
from copy import deepcopy
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score
from sklearn.preprocessing import StandardScaler
from scipy.optimize import linear_sum_assignment
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

from duped_modules.dcn_duped import DCNDuped
from duped_modules.dec_duped import DECDuped, IDECDuped
from duped_modules.embeddings_autoencoder import EmbeddingsAutoencoder
from load_datasets import load_all_datasets

In [2]:
def cluster_accuracy(y_true, y_pred):
    # We need to map the labels to our cluster labels
    # This is a linear assignment problem on a bipartite graph
    k = max(len(np.unique(y_true)), len(np.unique(y_pred)))
    cost_matrix = np.zeros((k, k))
    for i in range(y_true.shape[0]):
        cost_matrix[y_true[i], y_pred[i]] += 1
    inverted_cost_matrix = cost_matrix.max() - cost_matrix
    row_ind, col_ind = linear_sum_assignment(inverted_cost_matrix)
    return cost_matrix[row_ind, col_ind].sum() / y_pred.size

In [3]:
datasets = load_all_datasets(max_rows=5000)

accuracies = {d.name: {} for d in datasets}
nmis = {d.name: {} for d in datasets}